In [528]:
import re
import json
from pprint import pprint
import pandas as pd

In [529]:
with open('files/master.txt') as f:
    lines = f.readlines()

In [530]:
lines[1:100]


['\n',
 'Scott Lohbauer (he/him/his)  4:40 PM\n',
 'joined #wordle.\n',
 '\n',
 'Scott Lohbauer (he/him/his)  4:40 PM\n',
 'set the channel description: Room for Jornayans to share their Wordle scores\n',
 '\n',
 'Scott Lohbauer (he/him/his)  5:01 PM\n',
 'Wordle 205 5/6*\n',
 ':white_large_square::white_large_square::large_yellow_square::white_large_square::white_large_square:\n',
 ':white_large_square::large_yellow_square::white_large_square::large_yellow_square::white_large_square:\n',
 ':large_yellow_square::large_green_square::white_large_square::white_large_square::large_yellow_square:\n',
 ':white_large_square::large_green_square::white_large_square::large_yellow_square::large_yellow_square:\n',
 ':large_green_square::large_green_square::large_green_square::large_green_square::large_green_square:\n',
 '\n',
 'Scott Lohbauer (he/him/his)  3:33 PM\n',
 'Wordle 206 3/6*\n',
 ':white_large_square::white_large_square::white_large_square::large_yellow_square::large_yellow_square:\n',


In [531]:
def find_name(x):
    if re.match('(.*)[\(\)]? \d{1,2}:\d{1,2} [AP]M', x) != None and 'Today at' not in x and 'reply' not in x and 'replies' not in x:
        return x

def find_post(x):
    x = x.encode('unicode_escape').decode()
    if re.match('.*(Wordle \d{1,3} \d\/\d[\*]?)', x) != None:
        return re.match('.*(Wordle \d{1,3} \d\/\d[\*]?)', x)[1]

def find_squares(x):
    if '::' in x and 'square' in x:
        x = x.encode('unicode_escape').decode()

        s = re.match('.*(Wordle \d{1,3} \d\/\d[\*]?)(.*?)((:large_green_square:){5})', x).groups()

        s = ''.join(list(filter(lambda a: 'square' in a and a != ':large_green_square:', s))).strip()
        return s

msg =  'Wordle 206 3/6*\n:white_large_square::white_large_square::white_large_square::large_yellow_square::large_yellow_square:\n:white_large_square::large_green_square::large_green_square::large_green_square::white_large_square:\n:large_green_square::large_green_square::large_green_square::large_green_square::large_green_square:\n\nChris Shibe  12:51 PM\n'

m = 'Scott Lohbauer (he/him/his)  10:47 AM\noh boy\n10:48\nMe yesterday: I hate it\nWordle 221 4/6*\n:white_large_square::large_yellow_square::white_large_square::white_large_square::white_large_square:\n:large_yellow_square::white_large_square::white_large_square::white_large_square::white_large_square:\n:white_large_square::white_large_square::large_green_square::white_large_square::large_green_square:\n:large_green_square::large_green_square::large_green_square::large_green_square::large_green_square:\n:grimacing:\n1\n\n\nScott Lohbauer (he/him/his)  8:45 AM\n'
find_post(m)


'Wordle 221 4/6*'

In [538]:

posts_filtered = []
name = 'None'
msg = []
new_name = None
wordle = 'None None None'
new_wordle = None
squares = []
raw = []
for i,l in enumerate(['None None 00:00 AM'] + lines):
    msg.append(l)
    
    if find_name(l) != None: new_name = find_name(l)

    if (new_name != None and new_name != name):     
      msg = ''.join(msg)
      raw.append(name+msg)
      if find_post(msg) != None :
        squares = find_squares(msg)
        wordle = find_post(msg)
        tries = wordle.split(' ')[2].split('/')[0]
        posts_filtered.append({'name': re.sub('\(.*\) | \d{1,2}:\d{1,2} [AP]M.*', '', name).strip(),
                               'first_name': name.split(' ')[0],
                               'post': ''.join(name+msg),
                               'wordle': wordle.split(' ')[1],
                               'tries': tries,
                               'board': re.sub('\\\\n','\\n',squares),
                               'hardmode': True if '*' in wordle else False})

      name = new_name
      msg = []
    
# write out last game
if (new_name != None):     
      msg = ''.join(msg)
      raw.append(new_name+msg)
      if find_post(msg) != None :
        squares = find_squares(msg)
        wordle = find_post(msg)
        tries = wordle.split(' ')[2].split('/')[0]
        posts_filtered.append({'name': re.sub('\(.*\) | \d{1,2}:\d{1,2} [AP]M.*', '', name).strip(),
                               'first_name': new_name.split(' ')[0],
                               'post': ''.join(new_name+msg),
                               'wordle': wordle.split(' ')[1],
                               'tries': tries,
                               'board': re.sub('\\\\n','\\n',squares),
                               'hardmode': True if '*' in wordle else False})
  

posts_filtered

[{'name': 'Scott Lohbauer',
  'first_name': 'Scott',
  'post': 'Scott Lohbauer (he/him/his)  5:01 PM\nWordle 205 5/6*\n:white_large_square::white_large_square::large_yellow_square::white_large_square::white_large_square:\n:white_large_square::large_yellow_square::white_large_square::large_yellow_square::white_large_square:\n:large_yellow_square::large_green_square::white_large_square::white_large_square::large_yellow_square:\n:white_large_square::large_green_square::white_large_square::large_yellow_square::large_yellow_square:\n:large_green_square::large_green_square::large_green_square::large_green_square::large_green_square:\n\nScott Lohbauer (he/him/his)  3:33 PM\n',
  'wordle': '205',
  'tries': '5',
  'board': '\n:white_large_square::white_large_square::large_yellow_square::white_large_square::white_large_square:\n:white_large_square::large_yellow_square::white_large_square::large_yellow_square::white_large_square:\n:large_yellow_square::large_green_square::white_large_square::whi

In [539]:
raw

['NoneNone None 00:00 AM',
 'None None 00:00 AM\n\nScott Lohbauer (he/him/his)  4:40 PM\n',
 'Scott Lohbauer (he/him/his)  4:40 PM\njoined #wordle.\n\nScott Lohbauer (he/him/his)  4:40 PM\nset the channel description: Room for Jornayans to share their Wordle scores\n\nScott Lohbauer (he/him/his)  5:01 PM\n',
 'Scott Lohbauer (he/him/his)  5:01 PM\nWordle 205 5/6*\n:white_large_square::white_large_square::large_yellow_square::white_large_square::white_large_square:\n:white_large_square::large_yellow_square::white_large_square::large_yellow_square::white_large_square:\n:large_yellow_square::large_green_square::white_large_square::white_large_square::large_yellow_square:\n:white_large_square::large_green_square::white_large_square::large_yellow_square::large_yellow_square:\n:large_green_square::large_green_square::large_green_square::large_green_square::large_green_square:\n\nScott Lohbauer (he/him/his)  3:33 PM\n',
 'Scott Lohbauer (he/him/his)  3:33 PM\nWordle 206 3/6*\n:white_large_squ

In [540]:
with open('wordles.json', 'w') as f:
    json.dump(posts_filtered, f)